# Blue Team KubeHound Workflow

A step by step example workflow of how to use KubeHound for an incident response scenario.

## Initial Setup

Connection is being initated directly from the docker using the env vars `GRAPH_NOTEBOOK_HOST` and `GRAPH_NOTEBOOK_PORT`. To overwrite it you can use the magic `%%graph_notebook_config` [details here](https://github.com/aws/graph-notebook/tree/main/additional-databases/gremlin-server#connecting-to-a-local-gremlin-server-from-jupyter).

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

## Workflow

### Compromised Credentials

Let us consider a scenario where a user's credentials have been compromised. We can use KubeHound to identify the resources that the user has access to, whether any lead to critical assets and what attacks might have been leveraged.

First let's see whether there are any critical paths accessible. Because Kubernetes delegates the management of users' group memberships to third party components (e.g identity providers), we need to check paths from both the user and any groups they are a member of. 

**NOTE** the mapping of users to groups must be done prior to this step as it falls outside the scope of KubeHound and is specific to the identity provider.

In [ ]:
%%gremlin
kh.identities()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("type", "Group").has("name", within("dept-sales", "k8s-users")),
        has("type", "User").has("name", "bits.barkley@datadoghq.com"))
    .hasCriticalPath()
    .values("name")

Now let's examine the possible attack paths that could be taken by an attacker who has access to the compromised credentials.

In [ ]:

%%gremlin -d class -g critical -le 50 -p inv,oute
kh.identities()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("type", "Group").has("name", within("dept-sales", "k8s-users")),
        has("type", "User").has("name", "bits.barkley@datadoghq.com"))
    .criticalPaths()
    .by(elementMap())
    .limit(100)  // Limit the number of results for large clusters


Skip to the [next section](#advanced-workflows) to for more in-depth workflows to surface potential detection sources and eliminate attacks to narrow down the scope.

### Compromised Container

Consider the scenario where a container has been compromised via a malicious dependency found within a known set of images. We can use KubeHound to identify the resources that the container has access to, whether any lead to critical assets and what attacks might have been leveraged.

First let's see whether there are any critical paths accessible

In [ ]:
%%gremlin
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("image", TextP.containing("nginx")),    // Replace with your image name
        has("image", TextP.containing("cilium")))   // Replace with your image name
    .hasCriticalPath()
    .values("name")
    .dedup()

Now let's examine the possible attack paths that could be taken by the attacker.

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("image", TextP.containing("nginx")),    // Replace with your image name
        has("image", TextP.containing("cilium")))   // Replace with your image name
    .criticalPaths()
    .by(elementMap())
    .limit(100)  // Limit the number of results for large clusters

## Advanced Workflows

In a real-world deployment the above queries may throw up too many results to be actionable. In such cases we can use KubeHound to narrow down the scope of the investigation.

### Focus on container escapes

For example in the compromised container case we may wish to understand the easiest attack path that the attacker could have taken and focus our detections efforts there. Let's first look for any potential container escapes from the compromised container. These provide easy privilege escalation for an attacker but could also provide detection opportunities for us. The query below provides a list of container escapes possible from the matching images.

In [ ]:
%%gremlin
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .where(out().hasLabel("Node"))
    .or(
        has("image", TextP.containing("nginx")),    // Replace with your image name
        has("image", TextP.containing("cilium")))  // Replace with your image name
	.project('image',"escapes")
	.by(values("image"))
	.by(outE().where(inV().hasLabel("Node")).label().fold())
	.dedup()

### Shortest attack paths

Attackers are incentivized to take the shortest path to their target. We can use KubeHound to identify the shortest attack paths from the compromised container to the critical assets. This can help us focus our detection efforts on the most likely attack paths.

First we calculate the length of the shortest attack path from our target container to a critical asset:

In [ ]:
%%gremlin 
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("image", TextP.containing("nginx")),    // Replace with your image name
        has("image", TextP.containing("cilium")))   // Replace with your image name
    .minHopsToCritical()

Then we can find the unique attack paths of that length:

In [ ]:
%%gremlin -d class -g critical -le 50 -p inv,oute
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("image", TextP.containing("nginx")),    // Replace with your image name
        has("image", TextP.containing("cilium")))   // Replace with your image name
    .repeat(
      outE().inV().simplePath())
    .emit()
    .until(
        has("critical", true)
        .or().loops().is(4))
    .has("critical", true)
    .dedup()
    .path()
    .by(elementMap())

### Blast radius evaluation

It may be the case that the compromised container does not have a path to a critical asset (at least within the KubeHound model). In this case in can be useful to understand the blast radius of the compromised container. We can use KubeHound to identify all the resources that an attacker could have accessed from a compromised container.

In [ ]:
%%gremlin -d name -g class -le 50 -p inv,oute
kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .or(
        has("image", TextP.containing("nginx")),
        has("image", TextP.containing("cilium")))
   	.repeat(
      outE().inV().simplePath())
    .times(5) // Increase to expand the potential blast radius, but graph size will increase exponentially!
    .emit()
    .path()
    .by(elementMap())
    .limit(100)  // Limit the number of results for large clusters